In [ ]:
%tensorflow_version 1.x
import json
import numpy as np
#from sklearn.model_selection import train_test_split
import tensorflow.keras as keras
import matplotlib.pyplot as plt

from tensorflow.keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Input
from keras.layers.normalization import BatchNormalization
from tensorflow.keras import regularizers
from keras.optimizers import Adam, SGD
from keras.layers import LeakyReLU

In [ ]:
#Neccessary git clones


In [ ]:
#Loading in data
data_path = "./data_500_mini.json"
#Load data
with open(data_path, "r") as fp:
    data = json.load(fp)



In [ ]:
# data.keys()
labels=data['metadata']['all labels in loaded order']
# data['metadata'].keys()
#data['dataperclass'].keys()

In [ ]:
X=list(); Y=list()
C_x=list();C_y=list()
for i,c in enumerate(labels):
  print(c)
  skel_class=data['dataperclass'][c]['skelsinfo_list']
  #print(np.array(skel_class))
  C_x.append(skel_class)
  print(np.array(C_x).shape)
  label_class=data['dataperclass'][c]['label_list']
  C_y.append(label_class)
  #print(len(C_y))
  X.extend(skel_class)
  Y.extend(label_class)



follow
(1, 50, 24)
pour
(2, 50, 24)
pp
(3, 50, 24)
stand
(4, 50, 24)
stop
(5, 50, 24)
wave
(6, 50, 24)


In [ ]:
print(np.array(X).shape)
print(np.array(Y).shape)

(300, 24)
(300,)


In [ ]:
mapping={l:i for i,l in enumerate(labels)}
print(mapping)
labels=[]
for y in Y:
  labels.append(mapping[y])

{'follow': 0, 'pour': 1, 'pp': 2, 'stand': 3, 'stop': 4, 'wave': 5}


In [ ]:
encoder_Y = to_categorical(labels)
print(Y[0], ": ", encoder_Y[0])
print(Y[50], ": ", encoder_Y[50])
print(Y[100], ": ", encoder_Y[100])
print(Y[150], ": ", encoder_Y[150])
print(Y[200], ": ", encoder_Y[200])

follow :  [1. 0. 0. 0. 0. 0.]
pour :  [0. 1. 0. 0. 0. 0.]
pp :  [0. 0. 1. 0. 0. 0.]
stand :  [0. 0. 0. 1. 0. 0.]
stop :  [0. 0. 0. 0. 1. 0.]


In [ ]:
import numpy as np
def train_test_split(X_data,Y_data,val=0.2,test=0.1):
    X_data=np.array(X_data);Y_data=np.array(Y_data)
    # create random train/test split
    indices = list(range(X_data.shape[0]))
    train_split=1.0-(val+test)
    num_training_indices = int(train_split * X_data.shape[0])
    num_valid_indices = int(val * X_data.shape[0])
    #num_test_indices = int(train_split * X_data.shape[0])
    np.random.shuffle(indices)
    train_indices = indices[:num_training_indices]
    valid_indices=indices[num_training_indices:num_training_indices+num_valid_indices]
    test_indices = indices[num_training_indices+num_valid_indices:]

    # split the actual data
    X_data_train, X_data_val, X_data_test = X_data[train_indices], X_data[valid_indices], X_data[test_indices]
    Y_data_train, Y_data_val ,Y_data_test = Y_data[train_indices], Y_data[valid_indices], Y_data[test_indices]
    return (X_data_train, X_data_val, X_data_test, Y_data_train, Y_data_val, Y_data_test)


In [ ]:
from tensorflow.keras.utils import normalize
X=normalize(X)
X_train, X_val, X_test, Y_train, X_test, Y_test=train_test_split(X,encoder_Y)

In [ ]:
X_train[0].shape

(24,)

In [ ]:
# Build a simple DNN model with keras
model = Sequential()
#model.add(Input(shape=(24,)))
model.add(Dense(units=48, input_shape=(24,), activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(units=128, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
keras.layers.Dropout(0.2),
model.add(Dense(units=64, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
keras.layers.Dropout(0.2),
model.add(Dense(units=16, activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
model.add(BatchNormalization())
model.add(Dense(units=6, activation='softmax'))
#model.add(Dense(units=1,))

# Training
# optimiser: Adam with learning rate 0.0001
# loss: categorical_crossentropy for the matrix form matrix_Y
# metrics: accuracy is evaluated for the model
model.compile(optimizer=Adam(0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 48)                1200      
_________________________________________________________________
batch_normalization_9 (Batch (None, 48)                192       
_________________________________________________________________
dense_12 (Dense)             (None, 128)               6272      
_________________________________________________________________
batch_normalization_10 (Batc (None, 128)               512       
_________________________________________________________________
dense_13 (Dense)             (None, 64)                8256      
_________________________________________________________________
batch_normalization_11 (Batc (None, 64)                256       
_________________________________________________________________
dense_14 (Dense)             (None, 16)               

In [ ]:
# batch_size: number of samples per gradient update
# epochs: how many times to pass through the whole training set
# verbose: show one line for every completed epoch
history=model.fit(X_train, Y_train, batch_size=24, epochs=100, verbose=2, validation_data=(X_val, Y_val))
#model.summary()

Train on 210 samples, validate on 60 samples
Epoch 1/100
 - 1s - loss: 2.5909 - accuracy: 0.1286 - val_loss: 2.0052 - val_accuracy: 0.1667
Epoch 2/100
 - 0s - loss: 2.1550 - accuracy: 0.2762 - val_loss: 2.0063 - val_accuracy: 0.1667
Epoch 3/100
 - 0s - loss: 1.8395 - accuracy: 0.3762 - val_loss: 2.0083 - val_accuracy: 0.1333
Epoch 4/100
 - 0s - loss: 1.6051 - accuracy: 0.5048 - val_loss: 2.0112 - val_accuracy: 0.1833
Epoch 5/100
 - 0s - loss: 1.3947 - accuracy: 0.5952 - val_loss: 2.0147 - val_accuracy: 0.1833
Epoch 6/100
 - 0s - loss: 1.1886 - accuracy: 0.7190 - val_loss: 2.0187 - val_accuracy: 0.1833
Epoch 7/100
 - 0s - loss: 1.0391 - accuracy: 0.8000 - val_loss: 2.0238 - val_accuracy: 0.1833
Epoch 8/100
 - 0s - loss: 0.9140 - accuracy: 0.8571 - val_loss: 2.0290 - val_accuracy: 0.1833
Epoch 9/100
 - 0s - loss: 0.8419 - accuracy: 0.8762 - val_loss: 2.0335 - val_accuracy: 0.1833
Epoch 10/100
 - 0s - loss: 0.7826 - accuracy: 0.9000 - val_loss: 2.0378 - val_accuracy: 0.1833
Epoch 11/100
 

In [ ]:
#Evaluate on a test set
test_loss, test_acc = model.evaluate(X_test,Y_test, verbose=2)

print('\nTest accuracy:', test_acc)


Test accuracy: 1.0


In [ ]:
# Save the trained model
model.save('./action_recognition_mlp.h5')

In [ ]:
model.predict(X_test)
for i in 

array([[2.86830496e-03, 9.86109316e-01, 3.12390458e-03, 4.86621168e-03,
        2.08732253e-03, 9.45040723e-04],
       [2.57861009e-03, 3.89045221e-03, 9.73893583e-01, 9.33307037e-03,
        2.44309800e-03, 7.86119699e-03],
       [2.97154556e-03, 9.86587703e-01, 4.03046701e-03, 4.24968358e-03,
        1.49510556e-03, 6.65450818e-04],
       [6.06596330e-03, 4.05704556e-03, 1.16498757e-03, 1.35262841e-02,
        9.65379596e-01, 9.80612915e-03],
       [6.15591276e-03, 1.43273761e-02, 1.91315096e-02, 9.24100161e-01,
        3.21352407e-02, 4.14979644e-03],
       [1.20190112e-02, 2.80692917e-03, 9.36150458e-03, 2.40340456e-03,
        6.19117124e-03, 9.67217982e-01],
       [6.96006324e-03, 1.21154357e-02, 2.42109075e-02, 9.23848093e-01,
        2.85078809e-02, 4.35757777e-03],
       [5.27912099e-03, 1.32443598e-02, 1.79772768e-02, 9.28091407e-01,
        3.13008390e-02, 4.10707295e-03],
       [4.09296807e-03, 9.79252398e-01, 2.72114319e-03, 8.38650856e-03,
        4.43117088e-03, 

In [ ]:
X_test.shape

(30, 24)

In [ ]:
  history_dict = history.history
  history_dict.keys()

In [ ]:
def plot_history(history):
  history_dict = history.history
  keys=list(history_dict.keys())
  acc = history_dict['accuracy']
  val_acc = history_dict['val_accuracy']
  loss = history_dict['loss']
  val_loss = history_dict['val_loss']

  epochs = range(1, len(acc) + 1)

  # "bo" is for "blue dot"
  plt.plot(epochs, loss, 'r', label='Training loss')
  # b is for "solid blue line"
  plt.plot(epochs, val_loss, 'b', label='Validation loss')
  plt.title('Training and validation loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  plt.show()

  plt.plot(epochs, acc, 'r', label='Training acc')
  plt.plot(epochs, val_acc, 'b', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend(loc='lower right')

  plt.show()





In [ ]:
plot_history(history)